# WASSA2023

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dependencies

In [110]:
repo_path = "https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/"
branch = "main"

In [111]:
!pip install transformers -q
!pip install accelerate -U -q
!pip install datasets -q
!pip install torch-summary -q
!pip install graphviz -q
!pip install torchview -q
!pip install bertviz -q
!pip install NRCLex -q
!pip install textblob -q
!python -m textblob.download_corpora -q

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [112]:
utils_url = f"{repo_path}{branch}/utils.py"
evaluation_url = f"{repo_path}{branch}/evaluation.py"

import os
if os.path.exists("utils.py"):
  !rm "utils.py"
if os.path.exists("evaluation.py"):
  !rm "evaluation.py"

!wget {utils_url}
!wget {evaluation_url}

--2023-07-18 17:55:44--  https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38193 (37K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  37.30K  --.-KB/s    in 0.003s  

2023-07-18 17:55:44 (13.0 MB/s) - ‘utils.py’ saved [38193/38193]

--2023-07-18 17:55:44--  https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/main/evaluation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10675 (10K) [text/pl

In [113]:
EMO_json_path_train = f"{repo_path}{branch}/datasets/EMO23_lexicon_per_word_.json"
EMP_json_path_train = f"{repo_path}{branch}/datasets/EMP23_lexicon_per_word_.json"
EMO_json_path_test = f"{repo_path}{branch}/datasets/EMO23_lexicon_per_word_test.json"
EMP_json_path_test = f"{repo_path}{branch}/datasets/EMP23_lexicon_per_word_test.json"

!wget {EMO_json_path_train}
!wget {EMP_json_path_train}
!wget {EMO_json_path_test}
!wget {EMP_json_path_test}

--2023-07-18 17:55:44--  https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/main/datasets/EMO23_lexicon_per_word_.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14536741 (14M) [text/plain]
Saving to: ‘EMO23_lexicon_per_word_.json.1’

EMO23_lexicon_per_w 100%[===================>]  13.86M  87.5MB/s    in 0.2s    

2023-07-18 17:55:45 (87.5 MB/s) - ‘EMO23_lexicon_per_word_.json.1’ saved [14536741/14536741]

--2023-07-18 17:55:45--  https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/main/datasets/EMP23_lexicon_per_word_.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.

In [114]:
import json
import torch
import string
from torch import nn
from torch.nn import BCEWithLogitsLoss
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from transformers import TrainingArguments, Trainer, TrainerCallback, EarlyStoppingCallback
from transformers import BertPreTrainedModel, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
from torch import nn
from transformers import AutoConfig, BertForSequenceClassification, RobertaForSequenceClassification, RobertaModel
from transformers.modeling_outputs import SequenceClassifierOutput
from typing import Optional, Union, Tuple
from nrclex import NRCLex
import importlib
import sys
from torch.utils.data import Dataset
from utils import *
importlib.reload(sys.modules['utils'])

<module 'utils' from '/content/utils.py'>

In [115]:
# set CUDA if available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("======= CUDA Available =======")
else:
    device = torch.device('cpu')
    print("======= CUDA NOT Available, run on CPU =======")

======= CUDA Available =======


## Set up

In [116]:
EMOTIONS_NAMES = [
    'fear',
    'anger',
    'anticipation',
    'trust',
    'surprise',
    'positive',
    'negative',
    'sadness',
    'disgust',
    'joy',
    'hope'
]

config = {}

In [117]:
#@title Global Features
TASK = "EMO" #@param ["EMO", "EMP"]
binary_EMO = True #@param {type:"boolean"}
EMO_binary_emotion = "fear" #@param ["fear", "anger", "surprise", "sadness", "disgust", "joy", "hope", "neutral"]

In [118]:
#@title Number of labels
num_labels = 8 #@param {type:"integer"}
config['num_labels'] = num_labels
if binary_EMO:
  config['num_labels'] = 2

In [119]:
#@title Model Class
model_id = 'distil-1cls' #@param {type:"string"}
model_class_string = "RobertaPreTrainedModel" #@param ["BertPreTrainedModel", "RobertaPreTrainedModel"]
model_name = 'distilroberta-emotion' #@param ["bert-base", "roberta-base", "roberta-emotion", "distilroberta-emotion", "roberta-empathy"]

if model_class_string == "BertPreTrainedModel":
  model_class = BertForSequenceClassification
else:
  model_class = RobertaForSequenceClassification

config['model_id'] = model_id
config['model_class_string'] = model_class_string
config['model_name'] = model_name

In [169]:
#@title Global Features
emo_count_global = False #@param {type:"boolean"}
empathy_count_global = False #@param {type:"boolean"}
distress_count_global = False #@param {type:"boolean"}
bio_global = True #@param {type:"boolean"}
gold_emotions = False #@param {type:"boolean"}
gold_empathy = True #@param {type:"boolean"}
gold_distress = True #@param {type:"boolean"}

if TASK == "EMO":
  gold_emotions = False
if TASK =="EMP":
  gold_distress = False
  gold_empathy = False

global_features_names = []
emotions_count = ['fear_count',	'anger_count','anticipation_count',	'trust_count',
                 'positive_count',	'negative_count', 'surprise_count',
                 'sadness_count',	'disgust_count',	'joy_count',	'hope_count']
bio = ['gender', 'age', 'income', 'race', 'education']
if emo_count_global:
  for emo in emotions_count: global_features_names.append(emo)
if empathy_count_global: global_features_names.append('empathy_count')
if distress_count_global: global_features_names.append('distress_count')
if bio_global:
  for b in bio: global_features_names.append(b)

if gold_empathy: global_features_names.append('empathy')
if gold_distress: global_features_names.append('distress')

config['emo_count_global'] = emo_count_global
config['empathy_count_global'] = empathy_count_global
config['distress_count_global'] = distress_count_global
config['bio_global'] = bio_global
config['gold_emotions'] = gold_emotions
config['gold_empathy'] = gold_empathy
config['gold_distress'] = gold_distress
config['global_features_names'] = global_features_names


In [170]:
#@title Local Features
emo_count_local = False #@param {type:"boolean"}
empathy_count_local = False #@param {type:"boolean"}
distress_count_local = False #@param {type:"boolean"}

local_features_names = []
num_local_features = 0
if emo_count_local:
  local_features_names.append('emotions')
  num_local_features += len(EMOTIONS_NAMES)
if empathy_count_local:
  local_features_names.append('empathy')
  num_local_features += 1
if distress_count_local:
  local_features_names.append('distress')
  num_local_features += 1

if len(local_features_names) == 0: local_features_names = None

config['emo_count_local'] = emo_count_local
config['empathy_count_local'] = empathy_count_local
config['distress_count_local'] = distress_count_local
config['local_features_names'] = local_features_names

dim_extra_features = 0
if gold_emotions:
  dim_extra_features = len(global_features_names) + 8
else:
  dim_extra_features = len(global_features_names)

In [171]:
#@title Prompt
bio_prompt = True #@param {type:"boolean"}
bio_sep = 'after SEP' #@param ["after SEP", "before SEP"]
emo_prompt = False #@param {type:"boolean"}
emo_sep = 'before SEP' #@param ["after SEP", "before SEP"]
empathy_prompt = False #@param {type:"boolean"}
emp_sep = 'before SEP' #@param ["after SEP", "before SEP"]

prompt_names_before_SEP = []
prompt_names_after_SEP = []
if bio_prompt:
    if bio_sep=='before SEP':
        prompt_names_before_SEP.append('prompt_bio')
    else:
        prompt_names_after_SEP.append('prompt_bio')
if empathy_prompt:
    if emp_sep=='before SEP':
        prompt_names_before_SEP.append('prompt_emp')
    else:
        prompt_names_after_SEP.append('prompt_emp')
if emo_prompt:
    if emo_sep=='before SEP':
        prompt_names_before_SEP.append('prompt_emo')
    else:
        prompt_names_after_SEP.append('prompt_emo')

config['bio_prompt'] = bio_prompt
config['emo_prompt'] = emo_prompt
config['empathy_prompt'] = empathy_prompt
config['prompt_names_before_SEP'] = prompt_names_before_SEP
config['prompt_names_after_SEP'] = prompt_names_after_SEP

In [172]:
#@title Last CLSs
n_last_cls = 4 #@param {type:"integer"}
mean_last_cls = False #@param {type: "boolean"}
concat_local_features = False #@param {type: "boolean"}
config['n_last_cls'] = n_last_cls
config['mean_last_cls'] = mean_last_cls
config['concat_local_features'] = concat_local_features

if concat_local_features:
  dim_extra_features += num_local_features

config['dim_extra_features'] = dim_extra_features

In [173]:
#@title Model configuration
train_batch_size = 8 #@param {type:"integer"}
val_batch_size = 8 #@param {type:"integer"}
learning_rate = 5e-5 #@param {type:"number"}
weight_decay = 0.08 #@param {type:"number"}
epochs = 30 #@param {type:"integer"}
patience = 5 #@param {type:"integer"}
dropout = 0.3 #@param {type:"number"}
weighted_loss = "None" #@param ["None", "normalized_inverse", "balanced"]

model_type ={
  'distilroberta-emotion':'j-hartmann/emotion-english-distilroberta-base',
  'roberta-emotion':'j-hartmann/emotion-english-roberta-large',
  'roberta-empathy': 'bdotloh/roberta-base-empathy',
  'bert-base': 'bert-base-cased',
  'roberta-base' : 'roberta-base'
}

model_config = {
    'model_id': model_id,
    'tokenizer_name': model_type.get(model_name),
    'model_name': model_type.get(model_name),
    'train_batch_size': train_batch_size,
    'val_batch_size': val_batch_size,
    'learning_rate': learning_rate,
    'weight_decay': weight_decay,
    'epochs': epochs,
    'seed': 42,
    'patience': patience,
    'early_stopping_threshold': 0,
    'weighted_loss': weighted_loss,
    'dropout' : dropout
}



In [174]:
path_tosave = f"{TASK}_{model_config['model_id']}"
!mkdir $path_tosave

config.update(model_config)
write_dict_to_json(config, f"{path_tosave}/config.json")

print("\nCONFIGURATION")
for k,v in config.items():
  print(f"{k}: {v}")

mkdir: cannot create directory ‘EMO_distil-1cls’: File exists

CONFIGURATION
num_labels: 2
model_id: distil-1cls
model_class_string: RobertaPreTrainedModel
model_name: j-hartmann/emotion-english-distilroberta-base
emo_count_global: False
empathy_count_global: False
distress_count_global: False
bio_global: True
gold_emotions: False
gold_empathy: True
gold_distress: True
global_features_names: ['gender', 'age', 'income', 'race', 'education', 'empathy', 'distress']
emo_count_local: False
empathy_count_local: False
distress_count_local: False
local_features_names: None
bio_prompt: True
emo_prompt: False
empathy_prompt: False
prompt_names_before_SEP: []
prompt_names_after_SEP: ['prompt_bio']
n_last_cls: 4
mean_last_cls: False
concat_local_features: False
dim_extra_features: 7
tokenizer_name: j-hartmann/emotion-english-distilroberta-base
train_batch_size: 8
val_batch_size: 8
learning_rate: 5e-05
weight_decay: 0.08
epochs: 30
seed: 42
patience: 5
early_stopping_threshold: 0
weighted_loss: Non

## Dataset

### WASSA dataset

In [175]:
class WASSADataset(Dataset):

    def __init__(
        self,
        tokenizer,
        essay,
        essay_id,
        targets,
        prompt_before_SEP=None,
        prompt_after_SEP=None,
        EMP_lexicon = None,
        EMO_lexicon = None,
        global_features = None,
        local_emotions = False,
        local_empathy = False,
        local_distress = False,
        max_len=None
        ):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.essay = essay
        self.essay_id = essay_id
        self.targets = targets
        self.EMP_lexicon = EMP_lexicon
        self.EMO_lexicon = EMO_lexicon

        self.prompt_before_SEP = prompt_before_SEP
        self.prompt_after_SEP = prompt_after_SEP

        self.global_features = global_features
        self.local_emotions = local_emotions
        self.local_empathy = local_empathy
        self.local_distress = local_distress

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        essay = str(self.essay[index])

        essay_id = self.essay_id[index]

        prompt_before_SEP = ""
        if self.prompt_before_SEP is not None:
          for p in self.prompt_before_SEP[index]:
            prompt_before_SEP += " " + str(p)

        prompt_after_SEP = ""
        if self.prompt_after_SEP is not None:
          for p in self.prompt_after_SEP[index]:
            prompt_after_SEP += " " + str(p)

        text = essay
        text_pair = None
        if prompt_before_SEP != "":
          text += str(prompt_before_SEP)
        if prompt_after_SEP != "":
          text_pair = str(prompt_after_SEP)

        inputs = self.tokenizer.encode_plus(
            text=essay,
            text_pair=text_pair,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
            return_token_type_ids=True
        )

        item = {
          'input_ids': inputs['input_ids'].flatten(),
          'attention_mask': inputs['attention_mask'].flatten(),
          'token_type_ids': inputs["token_type_ids"].flatten()
        }

        if self.targets is not None:
          item['labels'] = torch.FloatTensor(self.targets[index])

        if self.global_features is not None:
          item['global_features'] = self.global_features[index]

        n_local_features = 0
        features_tokens_row = []
        if self.local_emotions:
            n_local_features += len(EMOTIONS_NAMES)
            for i in range(len(EMOTIONS_NAMES)):
                features_tokens_row.append(0)
        if self.local_empathy:
            n_local_features += 1
            features_tokens_row.append(4)
        if self.local_distress:
            n_local_features += 1
            features_tokens_row.append(0)

        if n_local_features > 0:
            features_tokens = np.full((self.tokenizer.model_max_length, n_local_features), features_tokens_row)
        else:
          features_tokens = None

        word_count=0
        first_char=True
        last_char_is_space=False
        for char_idx, char in enumerate(essay):
          token_idx = inputs.char_to_token(char_idx)
          if token_idx is None:
            if first_char: last_char_is_space=True
            if not last_char_is_space and not first_char:
              word_count+=1
              last_char_is_space=True
            continue
          elif last_char_is_space:
            last_char_is_space=False
          first_char=False

          j = 0
          if char not in string.punctuation:
            if self.local_emotions:
              for i, emo in enumerate(EMOTIONS_NAMES):
                features_tokens[token_idx][i] = self.EMO_lexicon[str(essay_id)][emo][word_count]
              j += len(EMOTIONS_NAMES)

            if self.local_empathy:
              features_tokens[token_idx][j] = self.EMP_lexicon[str(essay_id)]['empathy'][word_count]
              j += 1


            if self.local_distress:
              features_tokens[token_idx][j] = self.EMP_lexicon[str(essay_id)]['distress'][word_count]

        if features_tokens is not None:
            item['local_features'] = torch.FloatTensor(features_tokens)

        #item['return_dict']=True

        return item

Read dataframes

In [176]:
"""TRAIN_DATA = f"{repo_path}{branch}/datasets/WASSA_essay_level_internal_train_preproc_upsampled.tsv"
VAL_DATA = f"{repo_path}{branch}/datasets/WASSA_essay_level_internal_val_preproc.tsv"
DEV_DATA = f"{repo_path}{branch}/datasets/WASSA_essay_level_dev_preproc.tsv"
"""
if binary_EMO:
    TRAIN_DATA = f"{repo_path}{branch}/dataset_binary/WASSA23_essay_level_train_preproc_downsampled_{EMO_binary_emotion}.tsv"
    all_train_df = pd.read_csv(TRAIN_DATA, sep='\t')
    train_df, val_df = split_train_val(all_train_df, val_size=0.1)
else:
    TRAIN_DATA = f"{repo_path}{branch}/datasets/WASSA23_essay_level_internal_train_preproc.tsv"
    VAL_DATA = f"{repo_path}{branch}/datasets/WASSA23_essay_level_internal_val_preproc.tsv"
    train_df = pd.read_csv(TRAIN_DATA, sep='\t')
    val_df = pd.read_csv(VAL_DATA, sep='\t')

DEV_DATA = f"{repo_path}{branch}/datasets/WASSA23_essay_level_dev_preproc.tsv"
dev_df = pd.read_csv(DEV_DATA, sep='\t')

Encode targets

In [177]:
if TASK =="EMO":
  label_encoder = EmotionsLabelEncoder()
  label_encoder.fit(train_df.emotion)
  y_train = label_encoder.encode(train_df.emotion)
  y_val = label_encoder.encode(val_df.emotion)
  y_dev = label_encoder.encode(dev_df.emotion)

if TASK == "EMP":
  y_train = np.array(train_df[['empathy', 'distress']])
  y_val = np.array(val_df[['empathy', 'distress']])
  y_dev = np.array(dev_df[['empathy', 'distress']])

if binary_EMO:
  y_train = np.array(train_df.emotion_dataset).reshape(-1, 1)
  y_train_encoded = np.zeros((y_train.shape[0], 2))
  for i in range(y_train.shape[0]):
    if y_train[i]==0:
      y_train_encoded[i][0] = 1
      y_train_encoded[i][1] = 0
    else:
      y_train_encoded[i][0] = 0
      y_train_encoded[i][1] = 1
  y_train = y_train_encoded

  y_val = np.array(val_df.emotion_dataset).reshape(-1, 1)
  y_val_encoded = np.zeros((y_val.shape[0], 2))
  for i in range(y_val.shape[0]):
    if y_val[i]==0:
      y_val_encoded[i][0] = 1
      y_val_encoded[i][1] = 0
    else:
      y_val_encoded[i][0] = 0
      y_val_encoded[i][1] = 1
  y_val = y_val_encoded

  y_dev_encoded = []
  for emo in dev_df['emotion']:
    if EMO_binary_emotion in emo:
      y_dev_encoded.append([0.0,1.0])
    else:
      y_dev_encoded.append([1.0,0.0])
  y_dev_encoded = np.array(y_dev_encoded)
  y_dev = y_dev_encoded

Extra global features

In [178]:
global_features_train = None
global_features_val = None
global_features_dev = None

if len(config.get('global_features_names')) > 0:
  scaler = StandardScaler()
  scaler.fit(np.array(train_df[config.get('global_features_names')]))

  def standard_scalar_features(features):
    return scaler.transform(features)


  global_features_train =  standard_scalar_features(np.array(train_df[config.get('global_features_names')]))
  global_features_val =  standard_scalar_features(np.array(val_df[config.get('global_features_names')]))
  global_features_dev =  standard_scalar_features(np.array(dev_df[config.get('global_features_names')]))

  if TASK == "EMP":
    label_encoder = EmotionsLabelEncoder()
    label_encoder.fit(train_df.emotion)
    gold_emotions_train = label_encoder.encode(train_df.emotion)
    gold_emotions_val = label_encoder.encode(val_df.emotion)
    gold_emotions_dev = label_encoder.encode(dev_df.emotion)
    global_features_train = np.concatenate((global_features_train, gold_emotions_train), axis = 1)
    global_features_val = np.concatenate((global_features_val, gold_emotions_val), axis = 1)
    global_features_dev = np.concatenate((global_features_dev, gold_emotions_dev), axis = 1)

Prompt

In [179]:
prompt_before_SEP_train = None
prompt_after_SEP_train = None
prompt_before_SEP_val = None
prompt_after_SEP_val = None
prompt_before_SEP_dev = None
prompt_after_SEP_dev = None

if len(config.get('prompt_names_before_SEP')) > 0:
  prompt_before_SEP_train = np.array(train_df[config.get('prompt_names_before_SEP')])
  prompt_before_SEP_val = np.array(val_df[config.get('prompt_names_before_SEP')])
  prompt_before_SEP_dev = np.array(dev_df[config.get('prompt_names_before_SEP')])

if len(config.get('prompt_names_after_SEP')) > 0:
  prompt_after_SEP_train = np.array(train_df[config.get('prompt_names_after_SEP')])
  prompt_after_SEP_val = np.array(val_df[config.get('prompt_names_after_SEP')])
  prompt_after_SEP_dev = np.array(dev_df[config.get('prompt_names_after_SEP')])

Lexicons

In [180]:
EMO_lexicon_train_dict = None
EMP_lexicon_train_dict = None
EMO_lexicon_dev_dict = None
EMP_lexicon_dev_dict = None
EMO_lexicon_test_dict = None
EMP_lexicon_test_dict = None
if config.get('local_features_names') is not None:
  if 'emotions' in config.get('local_features_names'):
    with open("/content/EMO23_lexicon_per_word_.json") as json_file:
      EMO_lexicon_train_dict = json.load(json_file)
    with open("/content/EMO23_lexicon_per_word_test.json") as json_file:
      EMO_lexicon_test_dict = json.load(json_file)

  if 'empathy' in config.get('local_features_names') or 'distress' in config.get('local_features_names'):
    with open("/content/EMP23_lexicon_per_word_.json") as json_file:
      EMP_lexicon_train_dict = json.load(json_file)
    with open("/content/EMP23_lexicon_per_word_test.json") as json_file:
      EMP_lexicon_test_dict = json.load(json_file)

## Model

In [181]:
def get_loss_weights(y, method):
  if method == 'balanced':
    weights_train = y.shape[0] / (y.shape[1] * np.sum(y, axis=0))
  else:
    inverse_n_samples = 1 / np.sum(y, axis=0)
    sum_inverses = sum(inverse_n_samples)
    weights_train = inverse_n_samples / sum_inverses
  return torch.cuda.FloatTensor(weights_train)

In [182]:
loss_weights_train = None
if model_config.get('weighted_loss')!='None':
  loss_weights_train = get_loss_weights(y_train, model_config.get('weighted_loss'))

### Custom model

In [183]:
class ClassificationHead(nn.Module):
    #Head for sentence-level classification tasks.

    def __init__(self, config, dim_extra_features, hidden_layers_to_concat, classifier_dropout, local_features, mean_last_cls):
        super().__init__()
        self.local_features = local_features

        if mean_last_cls:
          total_dims = config.hidden_size + dim_extra_features
        else:
          total_dims = config.hidden_size*hidden_layers_to_concat + dim_extra_features
        if self.local_features:
          total_dims += config.hidden_size
        self.dense = nn.Linear(total_dims, total_dims)
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(total_dims, config.num_labels)

    def forward(self, features, **kwargs):
        features = features.to(torch.float32) # by default float32 is used as the dtype
        x = self.dense(features)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class CustomSequenceClassification(model_class):

    def __init__(self, config, dim_extra_features=0, model_class = None, local_features_names=None, loss_weights=None, n_last_cls = 1, mean_last_cls = False, concat_local_features = False):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        self.local_features_names = local_features_names
        self.loss_weights = loss_weights
        self.n_last_cls = n_last_cls
        self.mean_last_cls = mean_last_cls
        self.concat_local_features = concat_local_features
        self.model_class = model_class
        if self.model_class == "BertPreTrainedModel":
          self.bert = BertModel(config)

        if self.model_class == "RobertaPreTrainedModel":
          self.roberta = RobertaModel(config)

        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = ClassificationHead(config,
                                            dim_extra_features,
                                            n_last_cls,
                                            classifier_dropout,
                                            local_features_names is not None,
                                            mean_last_cls)
        self.post_init()


    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        local_features = None,
        global_features = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = True,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SequenceClassifierOutput]:

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.model_class == "BertPreTrainedModel":
          outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        if self.model_class == "RobertaPreTrainedModel":
          outputs = self.roberta(
              input_ids,
              attention_mask=attention_mask,
              token_type_ids=token_type_ids,
              position_ids=position_ids,
              head_mask=head_mask,
              inputs_embeds=inputs_embeds,
              output_attentions=output_attentions,
              output_hidden_states=output_hidden_states,
              return_dict=return_dict,
          )

        cls_tokens = []
        for i in range(1, self.n_last_cls + 1):
            cls_tokens.append(outputs.hidden_states[-1 * i][:, 0, :])
        if self.mean_last_cls:
          # average cls tokens
          output = torch.mean(torch.stack(cls_tokens), dim=0)
        else:
          # concat cls tokens
          output = torch.cat(cls_tokens, dim=1)

        output = self.dropout(output)

        if local_features is not None:
          tokens_output = outputs.last_hidden_state
          tokens_output = self.dropout(tokens_output)

          if self.concat_local_features:
            tokens_output = torch.cat((
                    tokens_output,
                    local_features.reshape(outputs.last_hidden_state.shape[0], outputs.last_hidden_state.shape[1], -1)),
                              dim=2)

          mask = torch.zeros_like(attention_mask)
          # unmask tokens with high or low empathy, high distress levels, or expressing at least one emotion
          j = 0
          if 'emotions' in self.local_features_names:
            emotion_values = local_features[:,:,:11]
            mask[emotion_values.sum(dim=-1)>=1] = 1.0
            j += 11

          if 'empathy' in self.local_features_names:
            empathy_values = local_features[:,:,j]
            mask[(empathy_values>5) | ((empathy_values<3) & (empathy_values>=1))] = 1.0
            j += 1

          if 'distress' in self.local_features_names:
            distress_values = local_features[:,:,j]
            mask[distress_values>4] = 1.0

          # mean pooling of unmasked tokens
          input_mask_expanded = mask.unsqueeze(-1).expand(tokens_output.size()).float()
          sum_embeddings = torch.sum(tokens_output * input_mask_expanded, 1)
          sum_mask = input_mask_expanded.sum(1)
          sum_mask = torch.clamp(sum_mask, min = 1e-9)
          tokens_output = sum_embeddings/sum_mask

          # concat pooled tokens lexically relevant with cls token
          output = torch.cat((output, tokens_output), dim=-1)


        if global_features is not None: # global
          output = torch.cat((output, global_features), dim=-1)

        logits = self.classifier(output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = nn.MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = nn.CrossEntropyLoss(weight = self.loss_weights)
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = nn.BCEWithLogitsLoss(weight = self.loss_weights)
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [184]:
class BinaryDataset(Dataset):


  def __init__(
  self,
  tokenizer,
  essay,
  targets,
  max_len=None
  ):
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.essay = essay
    self.targets = targets


  def __len__(self):
    return len(self.targets)


  def __getitem__(self, index):
    essay = str(self.essay[index])


    inputs = self.tokenizer.encode_plus(
    text=essay,
    add_special_tokens=True,
    max_length=self.max_len,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
    return_token_type_ids=True
    )


    item = {
    'input_ids': inputs['input_ids'].flatten(),
    'attention_mask': inputs['attention_mask'].flatten(),
    'token_type_ids': inputs["token_type_ids"].flatten()
    }

    if self.targets is not None:
      item['labels'] = torch.FloatTensor(self.targets[index])


    return item

In [185]:
tokenizer = AutoTokenizer.from_pretrained(config.get('tokenizer_name'), truncation=True)
tokenizer

RobertaTokenizerFast(name_or_path='j-hartmann/emotion-english-distilroberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [186]:
if TASK == "EMO":
  problem_type = "multi_label_classification"
if TASK == "EMP":
  problem_type = "regression"

model = AutoModelForSequenceClassification.from_pretrained(
  config.get('model_name'),
  problem_type = "multi_label_classification",
  num_labels = config.get('num_labels'),
  classifier_dropout = config.get('dropout'),
  ignore_mismatched_sizes=True)

model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

## Training

Prepare datasets

In [187]:
train_set = BinaryDataset(
    tokenizer=tokenizer,
    essay=train_df.essay,
    targets = y_train)

val_set = BinaryDataset(
    tokenizer=tokenizer,
    essay=val_df.essay,
    targets = y_val)

dev_set = BinaryDataset(
    tokenizer=tokenizer,
    essay=dev_df.essay,
    targets = y_dev)

Set up training

In [188]:
if TASK == "EMO":
  metric_for_val = "eval_macro_f1"
  compute_metrics_trainer = compute_EMO_metrics_trainer
if TASK == "EMP":
  metric_for_val = "eval_avg_pearson"
  compute_metrics_trainer = compute_EMP_metrics_trainer

In [189]:
train_arguments = TrainingArguments(
    output_dir=f"./{config.get('model_name')}",
    per_device_train_batch_size = config.get('train_batch_size'),
    per_device_eval_batch_size = config.get('val_batch_size'),
    num_train_epochs = config.get('epochs'),
    evaluation_strategy = "steps",
    save_strategy = "steps",
    logging_strategy = "steps",
    logging_steps = 150,
    eval_steps = 150,
    save_steps = 150,
    learning_rate=config.get('learning_rate'),
    weight_decay=config.get('weight_decay'),
    load_best_model_at_end=True,
    metric_for_best_model = "eval_loss",
    seed=config.get('seed'),
) # TODO: custom other params

In [190]:
trainer = Trainer(
    model=model,
    args=train_arguments,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_trainer
)

Callbacks

In [191]:
class TrainerLoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path

    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero: # whether this process is the main one in a distributed setting
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")

trainer.add_callback(EarlyStoppingCallback(
    early_stopping_patience = config.get('patience'),
    early_stopping_threshold = config.get('early_stopping_threshold')))

trainer.add_callback(TrainerLoggingCallback(config.get('model_id')+"_log.json"))

Start training

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Macro F1,Micro F1,Micro Jaccard,Macro Precision,Macro Recall,Micro Precision,Micro Recall,Sklearn Accuracy,Roc Auc Micro
150,0.399400,0.656792,0.834957,0.836066,0.718310,0.837691,0.834052,0.836066,0.836066,0.836066,0.836066


In [ ]:
# Access the training logs
train_logs = trainer.state.log_history

metrics = [list(log.keys())[:-5] for log in train_logs if log.get('eval_loss') is not None][0]

train_loss_values = []
eval_loss_values = []
eval_metrics_values = []
for _ in metrics:
  eval_metrics_values.append([])
train_epochs = []
eval_epochs = []

best_metric = 0
best_train_loss = float('inf')
best_eval_loss = float('inf')

for log in train_logs:

  if log.get(metric_for_val) is not None:

    if log.get(metric_for_val) > best_metric:
      best_metric = log.get(metric_for_val)
      best_steps = log.get('step')
      best_epoch_metric = log.get('epoch')
    if log.get('eval_loss') < best_eval_loss:
      best_eval_loss = log.get('eval_loss')
      best_epoch_eval_loss = log.get('epoch')

    for i, metric in enumerate(metrics):
      eval_metrics_values[i].append(log.get(metric))

    eval_loss_values.append(log.get('eval_loss'))
    eval_epochs.append(log.get('epoch'))

  if log.get('loss') is not None:
    if log.get('loss') < best_train_loss:
      best_train_loss = log.get('loss')

    train_loss_values.append(log.get('loss'))
    train_epochs.append(log.get('epoch'))


Plot losses

In [ ]:
fig_name = "losses"
plot_metric_curve(
    values = [train_loss_values, eval_loss_values],
    epochs = [train_epochs, eval_epochs],
    metrics = ["train loss", "eval loss"],
    title = fig_name,
    path = f"{path_tosave}/{fig_name}.png")

Plot metrics

In [ ]:
fig_name = "metrics"
plot_metric_curve(
    values = eval_metrics_values[1:],
    epochs = [eval_epochs for _ in eval_metrics_values[1:]],
    metrics = metrics[1:],
    title = fig_name,
    path = f"{path_tosave}/{fig_name}.png")

## Evaluation

In [ ]:
def predict_emotions(results, gold_emotions):

  binarized_predictions = np.where(results >= 0.5, 1, 0)

  for i, bin_pred in enumerate(binarized_predictions):
    if np.all(bin_pred==0):
      binarized_predictions[i][np.argmax(results[i])] = 1

  predicted_emotions = label_encoder.decode(binarized_predictions)
  return predicted_emotions


In [ ]:
print(trainer.state.best_model_checkpoint)

outs = trainer.predict(dev_set)

In [ ]:
if binary_EMO:
  predictions = np.argmax(outs.predictions, axis=1)
  predictions = np.where(predictions==0, f'no-{EMO_binary_emotion}', EMO_binary_emotion)
  golds = np.argmax(outs.label_ids, axis=1)
  golds = np.where(golds==0, f'no-{EMO_binary_emotion}', EMO_binary_emotion)
elif TASK == "EMO":
  golds = label_encoder.decode(outs.label_ids)
  predictions = predict_emotions(outs.predictions, golds)
if TASK == "EMP":
  golds = outs.label_ids
  predictions = outs.predictions

In [ ]:
path_predictions = f"{path_tosave}/predictions_{TASK}.tsv"
path_metrics = f"{path_tosave}/dev_metrics_{TASK}.json"

scores = {
    'train_loss': float(best_train_loss),
    'eval_loss': float(best_eval_loss)
}

write_predictions(predictions, path_predictions)
challenge_metrics = compute_metrics(golds=golds, predictions=predictions, task=TASK)
scores.update(challenge_metrics)
scores['best_metric'] = float(best_metric)
scores['best_epoch_metric'] = float(best_epoch_metric)
scores['best_epoch_eval_loss'] =  float(best_epoch_eval_loss)

write_dict_to_json(scores, path_metrics)

scores

In [ ]:
if TASK == "EMO":
  fig_name = "confusion_matrix"
  plot_confusion_matrix(golds=golds,
                        predictions=predictions,
                        path= f"{path_tosave}/{fig_name}.png"
                        )
  print("\n")
  fig_name = "confusion_matrix_per_emotions"
  plot_confusion_matrix_per_emotions(gold_emotions = golds,
                                     predicted_emotions = predictions,
                                     path= f"{path_tosave}/{fig_name}.png"
                                     )

if TASK == "EMP":
  fig_name = f"empathy_true_vs_predicted"
  plot_true_vs_predicted(golds = golds[:,0],
                        predictions = predictions[:,0],
                        path= f"{path_tosave}/{fig_name}.png"
                        )
  print("\n")
  fig_name = f"distress_true_vs_predicted"
  plot_true_vs_predicted(golds = golds[:,1],
                        predictions = predictions[:,1],
                        path= f"{path_tosave}/{fig_name}.png"
                        )
  print("\n")
  fig_name = f"abs_diff_true_vs_predicted"
  plot_abs_diff_emp(golds = golds,
                        predictions = predictions,
                        path= f"{path_tosave}/{fig_name}.png"
                        )

## Inference

In [ ]:
def load_checkpoint_and_compute_metrics(checkpoint_path, test_set):

  model = CustomSequenceClassification.from_pretrained(checkpoint_path,
                                                     problem_type = problem_type,
                                                     classifier_dropout = config.get('dropout'),
                                                     model_class = config.get('model_class_string'),
                                                     num_labels=config.get('num_labels'),
                                                     dim_extra_features = config.get('dim_extra_features'),
                                                     local_features_names = config.get('local_features_names'),
                                                     n_last_cls = config.get('n_last_cls'),
                                                     mean_last_cls = config.get('mean_last_cls'),
                                                     concat_local_features = config.get('concat_local_features'),
                                                     loss_weights = loss_weights_train,
                                                     ignore_mismatched_sizes=True
                                                    )
  trainer = Trainer(model=model)
  # Perform prediction using the loaded checkpoint
  outs = trainer.predict(test_set)

  if TASK == "EMO":
    golds = label_encoder.decode(outs.label_ids)
    predictions = predict_emotions(outs.predictions[0], golds)
  if TASK == "EMP":
    golds = outs.label_ids
    predictions = outs.predictions[0]

  path_predictions = f"{path_tosave}/predictions_{TASK}_best_metric.tsv"
  path_metrics = f"{path_tosave}/dev_metrics_{TASK}_best_metric.json"

  write_predictions(predictions, path_predictions)
  challenge_metrics = compute_metrics(golds=golds, predictions=predictions, task=TASK)

  write_dict_to_json(challenge_metrics, path_metrics)

  print(challenge_metrics)


In [ ]:
best_metric_path = f"/content/{config.get('model_name')}/checkpoint-{best_steps}"
load_checkpoint_and_compute_metrics(best_metric_path, dev_set)

## Save on Google Drive

In [ ]:
# move the best checkpoint in the folder with model id
best_model_path = trainer.state.best_model_checkpoint

!mv $best_model_path /content/$path_tosave

In [ ]:
# move the checkpoint with best metric in the folder with model id
!mv $best_metric_path /content/$path_tosave

In [ ]:
# move the results to personal drive
!mv /content/$path_tosave /content/drive/MyDrive/hlt